In [1]:
import torch
import pickle
import numpy as np
import matplotlib.pyplot as plt
import torch.backends.cudnn as cudnn
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader

from PIL import Image

import load_data

## Preprocess Data

In [2]:
# https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
train_file_lst = ['./cifar-10/data_batch_1', './cifar-10/data_batch_2', './cifar-10/data_batch_3', './cifar-10/data_batch_4']

val_file_lst = ['./cifar-10/data_batch_5']

test_file_lst = ['./cifar-10/test_batch']

In [3]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict



def get_data(file_lst):
    data_lst, label_lst = [], []
    for fname in file_lst:
            dic = unpickle(fname)
            label = np.array(dic[b'labels'])
            data = np.array( dic[b'data'])

            reconstructed_data = []
            for pixel in data:
                r, g, b = pixel[0:1024], pixel[1024:2048], pixel[2048:3072]
                r, g, b = r.reshape(32, 32, 1), g.reshape(32, 32, 1),  b.reshape(32, 32, 1)

                img = np.concatenate([r, g ,b], -1)
                img = Image.fromarray(img)
                img = img.resize((64, 64))
                img = np.array(img)
                img = img.reshape(3, 64, 64)
                reconstructed_data.append(img)

            data_lst.extend(reconstructed_data)
            label_lst.extend(label)

    return np.array(data_lst), np.array(label_lst)


In [4]:
x_train, y_train = get_data(train_file_lst)
x_test, y_test = get_data(test_file_lst)
x_val, y_val = get_data(val_file_lst)

In [5]:
x_train, x_test, x_val = torch.Tensor(x_train), torch.Tensor(x_test), torch.tensor(x_val)
y_train, y_test, y_val = torch.Tensor(y_train), torch.Tensor(y_test), torch.tensor(y_val)

In [6]:
train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)
val_dataset = TensorDataset(x_val, y_val)

kwargs = {}

train_dataLoader = DataLoader(train_dataset, shuffle = True, batch_size=128, **kwargs)
test_dataLoader = DataLoader(test_dataset, shuffle = True, batch_size=128, **kwargs)
val_dataLoader = DataLoader(val_dataset, shuffle = True, batch_size=128, **kwargs)

In [14]:
train_loader, test_loader = load_data.get_data()

Files already downloaded and verified
Files already downloaded and verified


## AlexNet model created

In [15]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)

Using cache found in C:\Users\55366/.cache\torch\hub\pytorch_vision_v0.10.0


In [16]:
cuda = False
if (torch.cuda.is_available()):
    print("activate cuda")
    cuda = True

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

if (cuda):
    torch.cuda.manual_seed_all(42)
    cudnn.benchmark = True
    model.cuda()
    criterion.cuda()

activate cuda


In [25]:
total = 0
predicted_label = []
for batch_idx, (train_data, train_label) in enumerate(train_loader):
    batch_size = train_data.size(0)
    total += batch_size

    train_label = train_label.type(torch.LongTensor)
    if(cuda):
        train_data, train_label = train_data.cuda(), train_label.cuda()

    optimizer.zero_grad()
    train_data, train_label = Variable(train_data), Variable(train_label)
    output = model(train_data)

    _, predicted = torch.max(output.data, 1)

    print(predicted.cpu().numpy())

    break


[152 684 660 376 675 901 335 438 676 856 339 373  59 482 291 821 623 675
 813 653 339 675 408  43 653 675 335 499  29  21 675 103 404 817 358 175
 167 241 675 188 856 339 381  73 190 175 675 158   1 339 596 126  43 385
  64 675 481 194 167 856 914 522 473 335 152 451 675 510 335 404 369 152
 167 175  80 901 371 912  59 439 503  68 675 152 335   5 708 491 384 404
 510 103  98 499 335  80 373 147 482 128 167 438 902 385 365 241 241 167
 335 404  98 149 510 856 408 914 158 510 335 675 675 712 375 147  38 152
 956 856]


In [20]:
print(predicted_label[0])

AttributeError: 'builtin_function_or_method' object has no attribute 'data'

In [9]:
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [10]:
'''model.classifier[4] = nn.Linear(4096,1024)
model.classifier[6] = nn.Linear(1024,10)
model.eval()'''

'model.classifier[4] = nn.Linear(4096,1024)\nmodel.classifier[6] = nn.Linear(1024,10)\nmodel.eval()'

## Training

In [11]:
def train(train, val, net, optimizer, criterion, cuda):
    net.train()
    total = 0

    train_corrected = 0
    total_loss = 0
    validation_iter = iter(val)

    for batch_idx, (train_data, train_label) in enumerate(train):
        batch_size = train_data.size(0)
        total += batch_size

        train_label = train_label.type(torch.LongTensor)
        if(cuda):
            train_data, train_label = train_data.cuda(), train_label.cuda()

        optimizer.zero_grad()
        train_data, train_label = Variable(train_data), Variable(train_label)
        output = net(train_data)

        loss = criterion(output, train_label)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(output.data, 1)
        
        train_corrected += predicted.eq(train_label.data).cpu().sum().item()

        total_loss += loss.item() * batch_size

    return total_loss/total, train_corrected/total

In [12]:
cuda = False
if (torch.cuda.is_available()):
    print("activate cuda")
    cuda = True

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

if (cuda):
    torch.cuda.manual_seed_all(42)
    cudnn.benchmark = True
    model.cuda()
    criterion.cuda()


activate cuda


In [13]:
for epoch in range(10):
    loss, train_acc = train(train_loader,test_loader, model, optimizer, criterion, True)
    print("e: %d, train_loss: %f, train_acc %f" % (epoch, loss, train_acc))

e: 0, train_loss: 1.826640, train_acc 0.356080
e: 1, train_loss: 1.183505, train_acc 0.575160
e: 2, train_loss: 0.936290, train_acc 0.673840
e: 3, train_loss: 0.815422, train_acc 0.717340
e: 4, train_loss: 0.723738, train_acc 0.749000
e: 5, train_loss: 0.657227, train_acc 0.776200
e: 6, train_loss: 0.599474, train_acc 0.795320
e: 7, train_loss: 0.569995, train_acc 0.805500
e: 8, train_loss: 0.539459, train_acc 0.817120
e: 9, train_loss: 0.506625, train_acc 0.827500
